977. Squares of a Sorted Array

https://leetcode.cn/problems/squares-of-a-sorted-array/

## 解法一：直接排序法 (O(N log N))

这是最直观的解法，先计算每个元素的平方，然后对结果进行排序。

### 写法 1

```python
# 直接排序法，O(NlogN) 的时间复杂度
class Solution:
    def sortedSquares(self, nums: List[int]) -> List[int]:
        return sorted(x * x for x in nums)
        # 推荐写法：代码简洁且空间效率最高
        # (x * x for x in nums) 是一个“生成器表达式”。当它作为函数的唯一参数时，外层的括号可以省略
        # sorted() 函数可以接收任意可迭代对象作为参数，但它在排序前，必须将所有元素加载到一个内部的临时列表中才能进行操作
        # 1. sorted(x * x for x in nums)  (当前写法 - 生成器)
        #    空间效率最高。此代码首先创建一个生成器对象（本身几乎不占内存）
        #    然后 sorted() 函数会迭代这个生成器，直接将元素逐个放入其内部用于排序的临时列表
        #    整个过程的峰值内存占用约为 O(N)，只创建了一个列表
        # 2. sorted([x * x for x in nums])  (列表推导式)
        #    空间效率较低。此代码分为两步：
        #    a) 列表推导式 [x * x for x in nums] 首先在内存中创建一个完整的、包含所有平方数的新列表（占用 O(N) 空间）
        #    b) 然后，sorted() 函数为了保证不修改原始数据，会再创建该列表的一个副本进行排序（又占用 O(N) 空间）
        #    整个过程的峰值内存占用约为 O(2N)
        # 3. sorted(list(x * x for x in nums))  (生成器转列表)
        #    与写法2在效果和内存占用上基本相同，但是先创建一个生成器，转换成列表，不推荐
```

### 写法 2 （不推荐）

```python
# 直接排序法写法二，改变原有列表，不推荐
class Solution1:
    def sortedSquares(self, nums: List[int]) -> List[int]:
        for i in range(len(nums)):
            nums[i] *= nums[i]
        nums.sort()
        return nums
```

### 写法 3

```python
# 直接排序写法三
class Solution2:
    def sortedSquares(self, nums: List[int]) -> List[int]:
        ans = []
        for num in nums:
            ans.append(num**2)
        ans.sort()
        # 列表的 .sort() 方法，只接受列表作为对象，会就地修改列表本身，使其有序，返回值是 None
        return ans
```

### 写法 4

```python
# 直接排序写法四
class Solution3:
    def sortedSquares(self, nums: List[int]) -> List[int]:
        new = [x * x for x in nums]
        new.sort()
        return new
        # list.sort() 改变原列表，sorted(list) 不改变列表
```

---

## 解法二：双指针法 (O(N))

利用原数组已经排序的特性，我们可以发现平方后的最大值一定出现在原数组的最左端或最右端。因此，我们可以使用双指针分别指向数组的头和尾，比较两者平方的大小，从后往前构建一个新数组。

### 写法 1：标准双指针

```python
# 双指针法，O(N) 的时间复杂度
class Solution4:
    def sortedSquares(self, nums: List[int]) -> List[int]:
        size = len(nums)
        left, right = 0, size - 1
        write_idx = size - 1
        results = [0] * size  # 这是生成全 0 列表的标准写法
        # results = [0 for _ in range(size)]，在这个简单场景下，第一种更常用

        while left <= right:
            # write_idx >= 0 也可以，等价条件，这里选择与后面一致的写法
            # 也可以直接 for write_idx in range(size - 1, -1, -1):，并且不需要 write_idx -= 1
            left_square = nums[left] * nums[left]
            right_square = nums[right] * nums[right]
            if left_square > right_square:
                results[write_idx] = left_square
                left += 1
            else:
                results[write_idx] = right_square
                right -= 1

            write_idx -= 1

        return results
```

### 写法 2：使用 append 后反转列表

```python
# 双指针法，最后反转列表，可以减少一个变量 write_idx，改用 append 方法添加元素，不需要知道索引
class Solution5:
    def sortedSquares(self, nums: List[int]) -> List[int]:
        # 根据 list 的先添加的元素排序在前原则
        # 将 nums 的平方按从大到小的顺序添加进新的 list
        # 最后反转 list
        results = []
        # 注意使用 append，就不能用 results = [0] * len(nums)，必须初始化空列表
        # 否则结果的 results 变成 2 * size 长
        left, right = 0, len(nums) - 1
        while left <= right:
            if abs(nums[left]) > abs(nums[right]):
                results.append(nums[left] ** 2)
                left += 1
            else:
                results.append(nums[right] ** 2)
                right -= 1
        return results[::-1]
```

### 写法 3：进一步优化

```python
# 进一步优化
class Solution6:
    def sortedSquares(self, nums: List[int]) -> List[int]:
        """
        整体思想：有序数组的绝对值最大值永远在两头，比较两头，平方大的插到新数组的最后
        1. 优化所有元素为非正或非负的情况
        2. 头尾平方的大小比较直接将头尾相加与 0 进行比较即可
        3. 新的平方排序数组的插入索引可以用倒序插入实现(使用 for 循环实现)
        """
        # 特殊情况, 元素都非负（优化1）
        if nums[0] >= 0:
            return [num**2 for num in nums]  # 按顺序平方即可
        # 最后一个非正，全负有序的
        if nums[-1] <= 0:
            return [num**2 for num in nums[::-1]]  # 倒序平方后的数组
        # 这里返回不能写 return (num**2 for num in nums)
        # 这样返回一个生成器对象，不是 list，会报错
        # 但返回 sorted(num**2 for num in nums) 是可以的
        # sorted 函数的参数虽然是生成器，但 sorted() 函数返回结果是 list
        # (x**2 for x in nums) 这部分（注意是圆括号）创建了一个“懒惰的”生成器，它本身不进行计算
        # sorted() 函数接收到这个生成器后，会把它“耗尽”——即，从生成器中取出所有的值（逐个计算平方），存储到一个临时列表里

        # 一般情况, 有正有负
        left = 0  # 原数组头索引
        right = len(nums) - 1  # 原数组尾部索引
        results = [0] * len(nums)  # 新建一个等长数组用于保存排序后的结果
        # write_idx = len(nums) - 1  # 新的排序数组尾插索引, 每次需要减一（优化 3 优化了）

        for write_idx in range(len(nums) - 1, -1, -1):
            # (优化3，倒序，不用单独创建变量，更加 pythonic 的写法)
            # if nums[left] ** 2 >= nums[right] ** 2:
            if nums[left] + nums[right] <= 0:  # (优化2)
                results[write_idx] = nums[left] ** 2
                left += 1
                # end_index -= 1  (优化3)
            else:
                results[write_idx] = nums[right] ** 2
                right -= 1
                # end_index -= 1  (优化3)
        return results
```

# Python sorted() 函数与 Timsort 算法核心思想总结

### 输入处理 (Input Handling)

`sorted()` 函数接受任何可迭代对象作为参数。它的第一步是立即将该可迭代对象物化为一个全新的内部列表。如果是生成器，则耗尽它；如果是列表，则创建一个副本。这个过程保证了原始数据不会被修改。

### 排序机制 (Sorting Mechanism)

#### 语义等价 (Semantic Equivalence)

在语义层面，`sorted(iterable)` 的行为效果等价于下列操作：

```python
# 概念上的等价代码
def equivalent_of_sorted(iterable, *, key=None, reverse=False):
    # 1. 将可迭代对象物化为新列表（不修改原数据）
    tmp = list(iterable)
    
    # 2. 在该新列表上就地排序
    tmp.sort(key=key, reverse=reverse)
    
    # 3. 返回这个被排序的新列表
    return tmp
```

### 排序机制（CPython，概览）

1. **物化（materialize）**
   - `sorted(iterable)` 首先将 `iterable` 物化为一个新的列表（不修改原数据），随后对该新列表进行内部排序。

2. **实现层面（C 层优化）**
   - 在 CPython 中，`sorted()`/`list.sort()` 的实际工作在 C 层完成；虽语义等价于 Python 层的 `list(iterable)` + `list.sort()`，但实现为更高效的 C 代码路径。

3. **装饰-排序-反装饰（DSU）**
   - 若传入 `key=`：
     1. CPython 在 C 层为要排序的每个元素调用一次 `key()`，将返回值与原值一起封装到一个“轻量包装对象”（由 C 实现的 `sortwrapper` / `kvpair` 类型，即 `PyObject*`）中并存入临时数组。`key` 对每个元素只调用一次，结果被缓存以避免重复计算。
     2. 对这些包装对象根据它们的 `key` 字段进行比较/排序（比较在 C 层执行以加速）。排序使用 CPython 的自适应自然合并策略（run detection + merge/galloping）。
     3. 排序完成后，从包装对象中提取原始 `value` 并把最终顺序的原始对象写回目标列表（并销毁包装对象）。实现上 CPython 是在 C 层的临时数组上操作并最后恢复到 Python 列表结构。

4. **算法（历史与现在）**
   - 历史：CPython 曾以 TimSort（由 Tim Peters 设计）为默认排序算法；该算法适应已有有序区（runs），并使用插入与合并策略以提高常见情况性能。  
   - 现代（CPython ≥ 3.11）：合并顺序的策略已改用 Powersort（对 run 合并顺序的策略更接近最优），因此官方实现可视为基于 Timsort 风格的自然合并，但合并策略已被 Powersort 改进。

### 设计权衡 (Design Trade-off)

这是一种典型的**“以空间换时间”**的工程决策。

* 通过一次性计算并缓存所有键，Python 避免了在排序比较过程中（可能多达 $O(N \log N)$ 次）反复调用 `key` 函数。

### 空间复杂度 (Space Complexity)

`sorted()` 的总空间开销主要包括三个部分：
1.  创建内部列表（输入数据的副本）所需的 $O(N)$ 空间。
2.  用于存储缓存键和元素位置信息的 DSU 结构所需的额外 $O(N)$ 空间。
3.  Timsort 临时内存：最坏 $O(N)$。

因此，其总空间复杂度为 $O(N)$。

### 核心算法 (Core Algorithm)

排序的核心是 **Timsort**。这是一种**混合、稳定、自适应**的排序算法。它智能地利用数据中可能存在的天然有序片段（runs），先通过插入排序等方式处理小的 run，再通过高效的归并策略将这些 run 合并，最终使整个列表变得有序。它对真实世界中常见的、部分有序的数据表现极其出色。

### 输出 (Output)

当 Timsort 算法排序完成后，那个被创建、装饰、排序并最终反装饰的内部列表，其本身就作为最终结果被函数返回。